# Árboles de decisión

Crearemos un arbol de decisión para predecil el salario `log(Salary)` de un jugador de béisbol, basado en el numero de años que lleva jugando en las grandes ligas `Years`, y el numero de batazos de hit `Hits` que hizo en el año anterior.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Leemos los datos:

In [ ]:
hitters = pd.read_csv('../data/hitters.csv').dropna()
hitters['LogSalary'] = np.log(hitters.Salary)
hitters = hitters[['Years', 'Hits', 'LogSalary']]
hitters.head()

### Recursive Binary Splitting
1.- Select the predictor $X_j$ and the cut point $s$ such that splitting the predictor space into regions $R_1(j,s) = \{X|X_j < s\}$ and $R_2 =(j,s) = \{X|X_j \geqslant s\}$ that leads to the greatest posible reduction in RSS.

$$\sum_{i: x_i\in R_1(j,s)} (y_i - \hat y_{R_1})^2 + \sum_{i: x_i\in R_2(j,s)} (y_i - \hat y_{R_2})^2 $$

In [ ]:
# Un árbol esta definido por la siguiente estructura
# {'left': None, 'right': None, 'df': df, 'rss': 0}

def _residual_squared_sum(df, y):
    """
    Calculate the residual squared sum in R, using the target y.
    """
    rss = 0
    yhat = 0
    return rss, yhat

_residual_squared_sum(hitters, 'LogSalary')

2.- We repeat the process, looking for the best predictor and best cutpoint in order to split the data further so as to minimize the RSS within each of the resulting regions. However, this time, instead of splitting the entire predictor space, we split one of the two previously identified regions. We now have three regions. Again, we look to split one of these three regions further, so as to minimize the RSS. The process continues until a stopping criterion is reached; for instance, we may continue until no region contains more than five observations.

In [ ]:
def _binary_splitting(df, y, x_j):
    """
    Find the best split using the predictor x_j. 
    """
    split = {
        'rss': 0,
        's': 1,
        'left': None,
        'right': None
    }
    return split

_binary_splitting(hitters, 'LogSalary', 'Years')

In [ ]:
def _recursive_binary_splitting(df, y, predictors):
    """
    Select the best variable to reduce the
    residual square sum from predictors.
    """
    return _binary_splitting(df, y, predictors[0])

_recursive_binary_splitting(hitters, 'LogSalary', ['Years', 'Hits'])

In [ ]:
def _make_tree(df, y):
    rss, yhat = _residual_squared_sum(df, y)
    return {'left': None, 'right':None, 
            'rss': rss,
            'yhat': yhat,
            'df': df,
            'y': y}

def _grow_tree(tree, predictors, min_points_per_leaf=5):
    """
    Recursively divide tree using the split that
    minimize rss. It stops when the region have
    less than 5 elements.
    """

hitters_tree = _make_tree(hitters, 'LogSalary')
_grow_tree(hitters_tree, ['Years', 'Hits'])

3.- Once the regions $R_1,...,R_J$ have been created,we predict the response for a given test observation using the mean of the training observations in the region to which that test observation belongs.

In [ ]:
def _evaluate(tree, event):
    yhat = 0
    return yhat

_evaluate(hitters_tree, hitters.iloc[0])

## Tree Pruning
Cost complexity:

For each value of $\alpha$ there corresponds a subtree $T \subset T_0$ such that

$$\sum_{m=1}^T \sum_{i: x_i \in R_m} (y_i - \hat y_{R_m})^2 + \alpha |T|$$

is as small as possible. $|T|$ indicates the number of terminal nodes of the tree $T$, $R_m$ is the rectangle (i.e. the subset of predictor space) corresponding to the $m$th terminal node, and $\hat y_{R_m}$ is the predicted response associated with $R_m$. 


1.- Calculate the total rss of a tree.

In [ ]:
def _is_leaf(tree):
    """
    Return if a given tree is a leaf.
    """
    return True

def _tree_rss(tree):
    """
    Calculate the total rss of a tree: the sum of the rss of
    all leaves.
    """
    rss = 0
    return rss

_tree_rss(hitters_tree)

2.- Count the number of leaves in the tree.

In [ ]:
def _count_leafs(tree):
    """
    Return the number of leaves in the tree.
    """
    leafs = 0
    return leafs

_count_leafs(hitters_tree)

3.- Calculate the cost complexity of the tree.

4.- Find the banch that if removed increase the tree rss the least.

In [ ]:
def _is_last_branch(tree):
    """
    Return if a given tree is the last branch, i.e. both 
    subtrees are leaves.
    """
    return True

def _find_min_deltarss(tree):
    """
    Find the tree that reduces for which the split reduces
    the least the rss.
    """
    min_deltarss = 0
    min_tree = tree
    return min_deltarss, min_tree

_find_min_deltarss(hitters_tree)

5.- Remove one by one the `min_deltarss` branch until the tree is a single leaf, while calculating the cost complexity. Return the tree that minimizes the cost complexity.

In [ ]:
from copy import deepcopy

def _prune_tree(tree, alpha):
    min_cc_tree = deepcopy(tree)
    return min_cc_tree

_prune_tree(hitters_tree, 0.1)

# Visualización

In [ ]:
import graphviz
from IPython import display

def make_node(g, name, tree):
    attributes = {'label': 'rss: {:0.2f}'.format(tree['rss']),
                  'shape': 'square'}
    if tree.get('y_hat'):
        attributes['label'] += '\n y_hat: {:0.2f}'.format(tree['y_hat'])
    g.node(name, **attributes)
    
def make_edge(g, parent, child, tree, direction):
    attributes = {}
    if direction == 'l':
        attributes['label'] = '{} < {}'.format(tree['x_j'], tree['s'])
    if direction == 'r':
        attributes['label'] = '{} >= {}'.format(tree['x_j'], tree['s'])
    g.edge(parent, child, **attributes)
    

def construct_tree(g, dna, tree):
    if not tree['left'] and not tree['right']:
        make_node(g, dna, tree)
    else:
        make_node(g, dna + 'l', tree['left'])
        make_edge(g, dna, dna + 'l', tree, 'l')
        construct_tree(g, dna + 'l', tree['left'])

        make_node(g, dna + 'r', tree['right'])
        make_edge(g, dna, dna + 'r', tree, 'r')
        construct_tree(g, dna + 'r', tree['right'])
        
def visualize_tree(tree):
    g = graphviz.Graph('tree', 'tree.dot', engine='dot')
    make_node(g, 'r', tree)
    construct_tree(g, 'r', tree)
    display.display(g)


In [ ]:
visualize_tree(hitters_tree)